In [1]:
%load_ext autoreload
%autoreload 2

import pickle, os
import numpy as np
import cvxpy as cp
from manipulation import Manipulation

In [2]:
mu_min, mu_max = 0.1, 0.1
h_min, h_max = 2, 2
h = 0.5*(h_min+h_max)
r_min, r_max = 1, 1
r = 0.5*(r_min+r_max)
w_std = 10
F_max = 1.

N_v = 5 #number of points to choose from
N_h = 6
num_grasps = 4

prob_params = [N_v, N_h, num_grasps, F_max, \
    h_min, h_max, h, \
    r_min, r_max, r, \
    mu_min, mu_max, w_std]

In [3]:
#setup filenames
relative_path = os.getcwd()
dataset_name = 'default'

if not os.path.isdir(os.path.join(relative_path, 'data', dataset_name)):
    os.mkdir(os.path.join(relative_path+'/data/'+dataset_name))
    
if not os.path.isdir(os.path.join(relative_path, 'config')):
    os.mkdir(os.path.join(relative_path, 'config'))

train_fn = os.path.join(relative_path, 'data', dataset_name, 'train.p')
test_fn = os.path.join(relative_path, 'data', dataset_name, 'test.p')

#load default parameter values
#define all possible params that can be varied
all_params = ['N_v', 'N_h', 'num_grasps', 'F_max', \
    'h_min', 'h_max', 'h', 'r_min', 'r_max', 'r', \
    'r_min', 'r_max', 'r', 'mu_min', 'mu_max', \
    'mu', 'w']

##TODO(pculbert): implement custom param variation.
#plan: split all_params into static & sampled params
#in Problem object, keep param dict with keys == all_params
#on constructing problem, make every param key map to either static value
#or a cvxpy.Parameter object.

### VARY ANY DESIRED PARAM VALUES HERE ###

param_dict = {'N_v': N_v, 'N_h': N_h, 'num_grasps': num_grasps, 'F_max': F_max, \
    'h_min': h_min, 'h_max': h_max, 'h': h, \
    'r_min': r_min, 'r_max': r_max, 'r': r, \
    'mu_min': mu_min, 'mu_max': mu_max, 'w_std': w_std}

#specify which parameters to sample, & their distributions
sampled_params = ['mu', 'w']
sample_dists = {'w' : lambda w: np.exp(w) / np.sum(np.exp(w)),  \
                'mu' : lambda : np.array([mu_min + (mu_max-mu_min)*np.random.rand()])}

#specify dataset sizes
num_train = 4500; num_test = 500
num_probs = num_train + num_test

#write out config
config_fn = os.path.join(relative_path, 'config', dataset_name+'.p')

config = [dataset_name, prob_params, sampled_params]
outfile = open(config_fn,"wb")
pickle.dump(config,outfile); outfile.close()

In [4]:
mp = Manipulation()

#create numpy containers for data: (params, x, u, y, J*, solve_time)
params = {}
if 'mu' in sampled_params:
    params['mu'] = np.zeros((num_probs,1))
if 'w' in sampled_params:
    params['w'] = np.zeros((num_probs,12))

Fs = np.zeros((num_probs, 3,12,N_v*N_h))
As = np.zeros((num_probs, 12))
Ws = np.zeros((num_probs, 12))
Y = np.zeros((num_probs, 1, N_v*N_h)).astype(int)

costs = np.zeros(num_probs); solve_times = np.zeros(num_probs)

costs = np.zeros(num_probs);
solve_times = np.zeros(num_probs)

In [ ]:
#big for loop, sampling problem data, solving MICP, & storing
ii = 0
while ii < num_probs:
    p_dict = {}
    if 'mu' in sampled_params:
        params['mu'][ii,:] = sample_dists['mu']()
        p_dict['mu'] = params['mu'][ii,:]
    if 'w' in sampled_params:
        params['w'][ii,:] = sample_dists['w'](np.random.rand(12))
        p_dict['w'] = params['w'][ii,:]

    prob_success = False
    try:
        prob_success, cost, solve_time, optvals = mp.solve_micp(p_dict)
    except (KeyboardInterrupt, SystemExit):
        raise
    except:
        print('mosek failed at '.format(ii))
    
    if prob_success:
        costs[ii] = cost; solve_times[ii] = solve_time
        As[ii,:], F_star, Y[ii,:,:] = optvals
        for jj in range(len(F_star)):
            Fs[ii,:,:,jj] = F_star[jj]
        ii += 1

In [ ]:
#post-processing + write out
train_params = {}; test_params = {}
if 'mu' in sampled_params:
    train_params['mu'] = params['mu'][:num_train,:]
    test_params['mu'] = params['mu'][num_train:,:]
if 'w' in sampled_params:
    train_params['w'] = params['w'][:num_train,:]
    test_params['w'] = params['w'][num_train:,:]
    
train_data = [train_params]
train_data += [As[:num_train,:], Fs[:num_train,:,:,:], Y[:num_train,:]]
train_data += [costs[:num_train], solve_times[:num_train]]

test_data = [test_params]
test_data += [As[num_train:,:], Fs[num_train:,:,:,:], Y[num_train:,:]]
test_data += [costs[num_train:], solve_times[num_train:]]

train_file = open(train_fn,'wb')
pickle.dump(train_data,train_file); train_file.close()

test_file = open(test_fn, 'wb')
pickle.dump(test_data,test_file); test_file.close()

In [ ]:
train_file = open(train_fn,'rb')
p, a, f, y, c, s = pickle.load(train_file); train_file.close()